# Student_psyco_project

* 微博手机端
https://m.weibo.cn/

## 爬虫代码

In [1]:
"""
author: Luzer
date: 2023/08/26
version: python 3, jupyter notebook
"""
import os # 常用包
import re # 正则表达式提取文本 !
from jsonpath import jsonpath
import requests # 发送请求
import pandas as pd # 存取csv文件
import datetime # 转换时间用
import time # 常用包

def trans_time(v_str):
    """转换时间"""
    GMT_FORMAT = '%a %b %d %H:%M:%S +0800 %Y'
    timeArray = datetime.datetime.strptime(v_str, GMT_FORMAT)
    ret_time = timeArray.strftime("%Y-%m-%d %H:%M:%S")
    return ret_time

def get_weibo_list(v_keyword,v_max_page):
    # 请求头，自己去定义
    # 如果爬爬不动了可以加入其他参数
    headers = {
        "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.54',
        "accept": "application/json, text/plain, */*",
        "accept-encoding": "gzip, deflate, br",
    }
    # 定义一个for循环，爬取不同页数的代码
    for page in range(1, v_max_page + 1):
        print('-------正在爬第{}页微博-------'.format(page))
        # 请求地址，从web端开发者模式里面找
        url = 'https://m.weibo.cn/api/container/getIndex'
        # 请求参数，从web端开发者模式里面找
        params = {
            "containerid": "100103type=1&q={}".format(v_keyword),
            "page_type": "searchall",
            "page": page,
        }
        # 发送请求
        r = requests.get(url, headers=headers, params=params)
        
        # 看下状态
        print("r.status_code: ", r.status_code) # 调试
        # 解析json数据
        cards = r.json()["data"]["cards"]
        # 微博内容
        text_list = []
        text_list = jsonpath(cards, '$..mblog.text') # 有语法结构的
        # 微博内容-正则：数据清洗
        dr = re.compile(r'<[^>]+>', re.S)
        text2_list = []
#         print('text_list is:') 调试
#         print(text_list)
        if not text_list: # 如果没有获取微博内容，进入下一轮循环
            continue
        if type(text_list) == list and len(text_list) > 0:
            for text in text_list:
                text2 = dr.sub('', text) # 正则表达式提取微博内容
                print(text2)
                text2_list.append(text2)
        # 微博创建时间
        time_list = jsonpath(cards, "$..mblog.created_at")
        time_list = [trans_time(v_str=i) for i in time_list]
        # 微博作者
        author_list = jsonpath(cards, '$..mblog.user.screen_name')
        # 微博ID
        id_list = jsonpath(cards, '$..mblog.id')
        # 微博b_id
        bid_list = jsonpath(cards, '$..mblog.bid')
        # 转发数
        reposts_count_list = jsonpath(cards, '$..mblog.reposts_count')
        # 评论数
        comment_count_list = jsonpath(cards, '$..mblog.comments_count')
        # 点赞数
        attitudes_count_list = jsonpath(cards, '$..mblog.attitudes_count')
        # 把列表数据保存程DF数据
        df = pd.DataFrame(
            {
                '页码': [page] * len(id_list),
                '微博ID': id_list,
                '微博BID': bid_list,
                '微博作者': author_list,
                '发布时间': time_list,
                '微博内容': text2_list,
                '转发数': reposts_count_list,
                '评论数': comment_count_list,
                '点赞数': attitudes_count_list,
            }
        )
        # 表头
        if os.path.exists(v_weibo_file):
            header = None
        else:
            header = ['页码', '微博ID', '微博BID', '微博作者','发布时间','微博内容','转发数','评论数','点赞数']
        # 保存到csv文件
        df.to_csv(v_weibo_file, mode='a+', index=False, header=header)
        print('Succeed to saving:{}'.format(v_weibo_file))




if __name__ == '__main__':
    # 爬取前几页
    max_search_page = 30  # 想爬取的页数
    # 爬取关键字
    search_keyword = '北京林业大学'# 这里是自己想搜索的关键字
    # 保存文件名
    v_weibo_file = '微博清单_{}_前{}页.csv'.format(search_keyword, max_search_page)

    # 如果csv文件存在，先删除之
    if os.path.exists(v_weibo_file):
        os.remove(v_weibo_file)
        print('微博清单存在，已删除: {}'.format(v_weibo_file))
    # 调用爬取微博函数
    get_weibo_list(v_keyword=search_keyword, v_max_page=max_search_page)
    
    # 数据清洗--去重
    df = pd.read_csv(v_weibo_file)
    # 删除重复数据
    df.drop_duplicates(subset=['微博BID'], inplace = True, keep = 'first')
    # 再次保存csv文件
    df.to_csv(v_weibo_file, index = False, encoding = 'utf_8_sig')
    print('数据清洗完成')




微博清单存在，已删除: 微博清单_北京林业大学_前30页.csv
===开始爬取第1页微博===

r.status_code:  200
新疆一位女孩考上大学，本是一件开心的事情，但家长却发文表示，“我死的心都有了，怎么办才能救自己！”  为什么让这位家长如此伤心呢？  据这位家长透露，原来其女儿今年高考成绩还不错，已经顺利被北京林业大学录取，但一看专业，瞬间就有些失落，胸口发闷，难受不已。  其女儿所录取的是“园林”专 ...全文
#妈妈去做英雄了很久不回来#【“妈妈去做英雄了，很长时间都不回来”】#门头沟区干部熊丽因公牺牲年仅36岁#  门头沟区王平镇在本轮强降雨中因抗洪救灾牺牲的干部熊丽今年36岁，江西宜春人，是两个孩子的母亲，其中女儿9岁，儿子2岁，熊丽父亲在其牺牲前一个月刚刚离世。  她于2006年考取北京林业大学， ...全文
中国地质大学（北京）、北京交通大学、北京科技大学、北京林业大学这四所北京“双一流”高校将疏解到河北省雄安新区，预计2025年秋季对外招生，2035年全部搬迁完毕。  都要搬走？ 
北京林业大学 有没有友友帮忙去隔壁汉华世纪大厦中公教育总部代登记一下本人退费信息 有偿🙋 
#北京林业大学“植绿报国”社会实践##我眼中的乡村#  我是“一村一品”品牌打造，助力王家山村产业振兴实践团，在湖北省十堰市郧阳区青曲镇王家山村展开乡村振兴实践活动@北林青春 
北京林业大学 公费医疗卡显示无效怎么办 
北京林业大学 求风景园林植物方向344和804考研资料 
#北京林业大学“植绿报国”社会实践##青春向党#  我们是519实践队，在北京市中国共产党历史展览馆和香山革命纪念馆，重温革命历史，深切缅怀革命先辈。 在这片百年征程的风景中，我们感受到了波澜壮阔的历史，也感悟到了百年初心的坚定。百年前，中国进入半殖民地半封建社会，人民深受苦难，但中国共产 ...全文
北京林业大学 求北林农村发展考研模拟题 
北京林业大学西区近邻宝能收哪些快递 
新改造的二食堂，环境是好了，但面积有限，到饭点人山人海，找座位都难，这还是新生没来的情况。 学校空间资源不足的问题，很明显。 北京林业大学#北林随手拍##北林大校园##食堂# 北京·北京林业大学 
北京林业大学求西区近邻宝电话 
北京林业大学 学二食堂的人也太多了叭除了装修，窗口基本都是原来的熟脸啊，大家怎么这么


r.status_code:  200
===开始爬取第15页微博===

r.status_code:  200
北京林业大学欢迎大家关注豆列-“北林图书馆可借”，一些自己喜欢的北林图书馆也有的书，最近图书馆买人文社科类的书变多了。 
北京林业大学 从🕖写到🕙的入学问卷 
北京林业大学 有家人知道未婚的研究生婚育证明必须要开吗 
北京林业大学 24考研为啥还不发大纲啊去年8月29都已经发了好慌好慌 
马克思主义理论 出北京林业大学马克思马理论思政考研真题、背诵笔记、题库、视频课，需要的滴滴！！！  
北京林业大学想问问大家，现在快递寄过去会帮忙送到寝室嘛？多大才送呀 
北京林业大学好心人们~请问研究生开学要自己带床垫子吗？宿舍床上是不是只有木板板呀 
北京林业大学 想问问预科的学长学姐宿舍是什么样的啊？ 
北京林业大学让你们知道我的存在 
#北京林业大学“植绿报国”社会实践##瑶里寻迹社会实践活动#  瑶里古镇 ·寻迹  前言   八月鸣蝉，正值炎炎夏日，瑶里寻迹小组在8月1号建军节开始了位于江西景德镇市瑶里村的探访之行，为了探寻当地文化历史与发展脉络，围绕着一个以产瓷器为主的小镇如何变成小有名气的旅游小镇主题，看到旅客和当地居 ...全文
Succeed to saving:微博清单_北京林业大学_前30页.csv
===开始爬取第16页微博===

r.status_code:  200
#北京林业大学“植绿报国”社会实践##瑶里寻迹社会实践活动#  瑶里古镇 ·寻迹  前言   八月鸣蝉，正值炎炎夏日，瑶里寻迹小组在8月1号建军节开始了位于江西景德镇市瑶里村的探访之行，为了探寻当地文化历史与发展脉络，围绕着一个以产瓷器为主的小镇如何变成小有名气的旅游小镇主题，看到旅客和当地居 ...全文
#北京林业大学“植绿报国”社会实践##国家资助讲啊讲社会实践# 我是“国家资助讲啊讲小队”，在宣传国家资助政策体系 快来看看有没有你不知道的资助内容吧！ @北林青春 
《2024年北京林业大学野生动植物保护与利用考研成功上岸经验分享》 北京林业大学考研 #野生动植物保护# 考研  2024年北京林业大学野生动植物保护与利用考研成功上岸经验分享 
《2024年北京林业大学企业管理考研经验分享》 北京林业大学考研 企业管理 考研  2024年北京林业大学企业管理考研经验分享 


r.status_code:  200
#北京林业大学“植绿报国”社会实践##“青春向党”感悟祖国发展伟力实践专项#  我是五幺四好运常在实践团队  在福建省陈埭民族中学 今天我参观了福建省泉州市晋江市陈埭民族中学，并进行了访谈，感悟了时代征程与祖国的发展伟力，这是一场让我受益匪浅的“青春向党”实践活动。陈埭民族中学对于青春向 ...全文
【23fall录取】#英国留学# Offer：伦敦大学学院 比较经济学与政策文学硕士 （UCL MA Comparative Economics and Policy ） BG：北京林业大学 国际经济与贸易，大四，GPA90.89，雅思7.5，GRE318 经历：👇评论 
#北京林业大学“植绿报国”社会实践##A4210环保实践行动#  我们是乡见团队湘西支队，我们在湖南湘西甘溪村，开展了以“铸传统智慧之魂，赋人居可续新生”为主题的实践活动。致力于发掘传统人居智慧，加强传统聚落保护，推广绿色科普知识，开展参与合作绿色营造，向公众传播魅力乡村！  @北林青春 
北京林业大学请问有工学院的研究生师兄师姐吗，研0新生想请教一些问题，有偿，拜托了🙏 
北京林业大学有人出山地车吗 
#北京林业大学“植绿报国”社会实践##A4210环保实践行动# 我们是阳光小队实践团，在河北省石家庄市，为深入开展学习贯彻习近平新时代中国特色社会主义思想主题教育，深入贯彻落实党的二十大精神，落实立德树人根本任务，进一步加大学生资助政策宣传力度，深化资助育人实效，落实《关于印发 2023 年学生 ...全文
#北京林业大学“植绿报国”社会实践# 我们是追溯红色记忆实践团，在山东省烟台市东炮台景区，深刻感受这座城市的厚重历史和文化底蕴，加深对历史文化遗产、和平稳定、个人责任与担当等方面的认识。 东炮台是一座具有重要历史意义的军事防御工事，它见证了明清两代沿海防御体系中的重要组成部分。这座城 ...全文
北京林业大学 求问学长学姐！803森林生态学两本参考书任背一本都可以吗？两本书表述上有些不一样，有些分类也不一样，不知道应该背哪本的 
Succeed to saving:微博清单_北京林业大学_前30页.csv
===开始爬取第30页微博===

r.status_code:  200
北京林业大学研0，电动车能进学校吗 
北京林业大学 本人是23届上岸工学院的